In [11]:
import cv2
import numpy as np
from stackImages import stackImages

In [12]:
def empty(a):
    pass

In [13]:
def findEdgePoints(img):
    cnts, _ = cv2.findContours(img, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_NONE)
    for cnt in cnts:
        area = cv2.contourArea(cnt)
        print("area: ",area)
        if area > 1000:
            peri = cv2.arcLength(cnt, True)
            approx = cv2.approxPolyDP(cnt, 0.03 * peri, True)
            print(approx.shape)
            print("approx: ", approx)
            cv2.drawContours(imgCnt, approx, -1, (255, 0, 0), 5)
            points = approx.reshape(approx.shape[0], 2).tolist()
            print("points: ", points)
            return points


In [14]:
from numpy import ndarray


def arrange_points(p) -> ndarray:
    if p == None: return np.array([])
    points = np.array(p)

    s = points.sum(axis=1)
    diff = np.diff(points, axis=1).ravel()

    top_left = points[np.argmin(s)]
    bottom_right = points[np.argmax(s)]
    top_right = points[np.argmin(diff)]
    bottom_left = points[np.argmax(diff)]

    return np.array([top_left, top_right, bottom_left, bottom_right]).tolist()

In [15]:
def findSideLength(points):
    arranged_points = arrange_points(points)
    top_left, top_right, bottom_left, bottom_right = arranged_points
    sisi_atas = np.sqrt((top_left[0] - top_right[0]) ** 2 + (top_left[1] - top_right[1]) ** 2)
    sisi_bawah = np.sqrt((bottom_left[0] - bottom_right[0]) ** 2 + (bottom_left[1] - bottom_right[1]) ** 2)
    sisi_kanan = np.sqrt((top_right[0] - bottom_right[0]) ** 2 + (top_right[1] - bottom_right[1]) ** 2)
    sisi_kiri = np.sqrt((top_left[0] - bottom_left[0]) ** 2 + (top_left[1] - bottom_left[1]) ** 2)

    return [sisi_atas, sisi_kiri, sisi_bawah, sisi_kanan]

In [16]:
def getWarp(img, points):
    sisi_atas, sisi_kiri, sisi_bawah, sisi_kanan = findSideLength(points)
    widthImg = int(round(np.average([sisi_atas, sisi_bawah])))
    heightImg = int(round(np.average([sisi_kanan, sisi_kiri])))
    pts1 = np.float32(points)
    pts2 = np.float32(
        [
            [0, 0],
            [widthImg, 0],
            [0, heightImg],
            [widthImg, heightImg]
        ]
    )
    print("width and height img: ",widthImg, heightImg)
    matrix = cv2.getPerspectiveTransform(pts1, pts2)
    warpp = cv2.warpPerspective(img, matrix, (widthImg, heightImg))
    return warpp

In [17]:
def greenScreen(cam):
    imgHSV = cv2.cvtColor(cam, cv2.COLOR_BGR2HSV)
    h_min = cv2.getTrackbarPos("Hue Min", "TrackBars")
    h_max = cv2.getTrackbarPos("Hue Max", "TrackBars")
    s_min = cv2.getTrackbarPos("Sat Min", "TrackBars")
    s_max = cv2.getTrackbarPos("Sat Max", "TrackBars")
    v_min = cv2.getTrackbarPos("Val Min", "TrackBars")
    v_max = cv2.getTrackbarPos("Val Max", "TrackBars")
    lower = np.array([h_min, s_min, v_min])
    upper = np.array([h_max, s_max, v_max])
    mask = cv2.inRange(imgHSV, lower, upper)
    inverted_mask = cv2.bitwise_not(mask)
    greenscreen = cv2.bitwise_and(cam, cam, mask=inverted_mask)

    return greenscreen, mask

In [18]:
def preProcessing(img):
    imgGray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    imgBlur = cv2.GaussianBlur(imgGray, (5, 5), 1)
    imgCanny = cv2.Canny(imgBlur, 200, 200)
    imgDilate = cv2.dilate(imgCanny, np.ones((5, 5)), iterations=3)
    imgErode = cv2.erode(imgDilate, np.ones((5, 5)), iterations=1)
    return imgErode

In [19]:
cv2.namedWindow("TrackBars")
cv2.resizeWindow("TrackBars", 640, 240)
cv2.createTrackbar("Hue Min", "TrackBars", 0, 179, empty)
cv2.createTrackbar("Hue Max", "TrackBars", 179, 179, empty)
cv2.createTrackbar("Sat Min", "TrackBars", 0, 255, empty)
cv2.createTrackbar("Sat Max", "TrackBars", 255, 255, empty)
cv2.createTrackbar("Val Min", "TrackBars", 0, 255, empty)
cv2.createTrackbar("Val Max", "TrackBars", 255, 255, empty)

In [20]:
cap = cv2.VideoCapture(1)


while True:
    _, cam = cap.read()
    imgCnt = cam.copy()
    greenscreen, mask = greenScreen(cam)
    preprocessing = preProcessing(greenscreen)
    
    camEdge = findEdgePoints(preprocessing)
    # warp = getWarp(cam, camEdge)
    cv2.imshow("Result", stackImages(0.3, ([cam, preprocessing], [greenscreen])))
    # cv2.imshow("WARP", warp)
    if cv2.waitKey(1) & 0xff == ord('q'):
        break

cv2.destroyAllWindows()